# Comapring polynomial equations

In [ ]:
import os       # using operating system dependent functionality (folders)
import glob
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
# import copy     # Can Copy and Deepcopy files so original file is untouched.
# from ipywidgets import IntSlider, Output
import ipywidgets as widgets
# from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(0, '../') # path to functions
from cvasl import file_handler as fh # 
from cvasl import mold #
from cvasl import carve
from cvasl import seperated
from cvasl.file_handler import Config

In [ ]:
#from scipy.optimize import curve_fit

## let's look through the top and stroke MRI datasets and derive our optimal second degree equasions.

In [ ]:
config = Config()
root_mri_directory = config.get_directory('raw_data')

In [ ]:
top = os.path.join(root_mri_directory, 'assembled/top_stitched.csv')
stroke_mri = os.path.join(root_mri_directory, 'assembled/StrokeMRI_stitched.csv')

In [ ]:
dataframe_top = pd.read_csv(top)
dataframe_stroke_mri = pd.read_csv(stroke_mri)

In [ ]:
dataframe_top.columns

In [ ]:
seperated.polyfit_and_show(
        dataframe_top,
        'Age',
        'GM_vol_Liter',
        2,
        color1='purple',
)

# Now we derive some facts about the distribution

Our x axis is age, our y axis is the dependant variable. We want to ask how much this dependant variable varies from the derived polynomial

First, we want to say for each point what is the difference in y on the polynomial versus y on reality

In [ ]:
def derived_function(column,a,b,c):
    list_output = []
    for value in column:
        output = a * (value**2) + b* value + c
        list_output.append(output)
    return list_output

In [ ]:
estimated = derived_function(dataframe_top['Age'], -1.18633655e-05, -2.15086048e-03,  7.95379583e-01)

In [ ]:
dataframe_for_difference_top = dataframe_top[['Age', 'GM_vol_Liter']]
dataframe_for_difference_top['predicted'] = estimated
dataframe_for_difference_top.sort_values(by=['Age'])

In [ ]:
dataframe_for_difference_top['differences'] = dataframe_for_difference_top['predicted'] - dataframe_for_difference_top['GM_vol_Liter']
dataframe_for_difference_top['absolute_differences'] = abs(dataframe_for_difference_top['differences'])
dataframe_for_difference_top.describe()

So we see our mean absolute diference and max difference from this ideal polynomial i.e. .046591 and 0.224849.We also see other aspects of distribution.

## Now here comes the fun.. we can ask if the polynomial will work for the other dataset.

In [ ]:
estimated_stroke_mri = derived_function(dataframe_stroke_mri['Age'], -1.18633655e-05, -2.15086048e-03,  7.95379583e-01)

In [ ]:
#dataframe_stroke_mri

In [ ]:
dataframe_for_difference_mri = dataframe_stroke_mri[['Age', 'GM_vol']]
dataframe_for_difference_mri['predicted'] = estimated_stroke_mri
dataframe_for_difference_mri.sort_values(by=['Age'])

In [ ]:
dataframe_for_difference_mri['differences'] = dataframe_for_difference_mri['predicted'] - dataframe_for_difference_mri['GM_vol']
dataframe_for_difference_mri['absolute_differences'] = abs(dataframe_for_difference_mri['differences'])
dataframe_for_difference_mri.describe()

So our max difference is actually lower...So in this case one "fits " within the distribution of the other

Now we need to do this for every column, and this is R1 